## Hello world Jupyter Gateway!

A very brief introduction to what you can do with jupyter gateway

In [1]:
# GET /text
print("hello world !")

hello world !


### JSON response

In [2]:
import json

In [3]:
# GET /json
print(json.dumps({"hello":"world"}))

{"hello": "world"}


In [4]:
# ResponseInfo GET /json
print(json.dumps({"headers" : { "Content-Type" : 'application/json'}, "status" : 400}))

{"headers": {"Content-Type": "application/json"}, "status": 400}


### RESTful API: counters

The Jupyter Kernel eventually is responsible for running the notebook cells that you are reading here. Next to executing the code cells, it will also keep as part of its state and variable that you will declare in the notebook. Here below we will use the context of the kernel to manage a dynamic list of counters via a RESTful API.

In [5]:
counters = dict()

Here below we are going to mock a REQUEST object as passed by the Jupyter Kernel Gateway, so that we can check if the code is functioning even when we are in jupyter notebook mode. When working in gateway mode, the REQUEST variable is provided by the client via the Gateway.

In [37]:
REQUEST = json.dumps({
    'body': {
        'value': 5
    },
    'path': {
        'id': 'cnt'
    },
    'args': {
    },
    'headers': {
    }
})

REQUEST

'{"body": {"value": 5}, "path": {"id": "cnt"}, "args": {}, "headers": {}}'

In [40]:
# PUT /counters/:id
req = json.loads(REQUEST)
counter_id = req['path']['id']
counter_value = req['body']['value']
counters[counter_id]=counter_value

In [41]:
# GET /counters/:id
req = json.loads(REQUEST)
counter_id = req['path']['id']

status = 200 if counter_id in counters else 404
value  = counters.get(counter_id, None)
if value:
    print(json.dumps({'value': value}))

{"value": 5}


In [42]:
# ResponseInfo GET /counters/:id
print(json.dumps({"headers" : { "Content-Type" : 'application/json'}, "status" : status}))

{"headers": {"Content-Type": "application/json"}, "status": 200}


In [52]:
# GET /counters
req = json.loads(REQUEST)
# query args appear as a list since they can be repeated in the URL
f = req.get('args', {}).get('filter', [None])[0]

if f:
    print(json.dumps([{k: {'value':v}} for k,v in counters.items() if f in k]))
else:
    print(json.dumps([{k: {'value':v}} for k,v in counters.items()]))

[]


In [48]:
# ResponseInfo GET /counters
print(json.dumps({"headers" : { "Content-Type" : 'application/json'}, "status" : 200}))

{"headers": {"Content-Type": "application/json"}, "status": 200}


In [49]:
# POST /counters/:id
req = json.loads(REQUEST)
counter_id = req['path']['id']

counters[counter_id]  = counters.get(counter_id, 0) + 1

In [50]:
# ResponseInfo POST /counters/:id
print(json.dumps({"status" : 201}))

{"status": 201}


In [51]:
# DELETE /counters/:id
req = json.loads(REQUEST)
counter_id = req['path']['id']
result = counters.pop(counter_id, None)

## Data API: supporting multiple data types
How to pass various data types from via a RESTfull API. https://en.wikipedia.org/wiki/Media_type

In [31]:
png = "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAUAAAAFCAYAAACNbyblAAAAHElEQVQI12P4//8/w38GIAXDIBKE0DHxgljNBAAO9TXL0Y4OHwAAAABJRU5ErkJggg=="

#### Inline: display it inline in the notebook

In [32]:
import base64
import IPython.display as display
display.Image(base64.b64decode(png.split(',')[1]))

#### Expose as a data field
This format works well for javascript background api calls

In [33]:
# GET /image-data
print(png)

data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAUAAAAFCAYAAACNbyblAAAAHElEQVQI12P4//8/w38GIAXDIBKE0DHxgljNBAAO9TXL0Y4OHwAAAABJRU5ErkJggg==


#### Expose as a html snippet
This format works well for inclusion in html dynamically generated views

In [34]:
# GET /image-html
print('<img src={} />'.format(png))

<img src=data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAUAAAAFCAYAAACNbyblAAAAHElEQVQI12P4//8/w38GIAXDIBKE0DHxgljNBAAO9TXL0Y4OHwAAAABJRU5ErkJggg== />


In [35]:
# ResponseInfo GET /image-html
print(json.dumps({"headers" : { "Content-Type" : 'text/html'}}))

{"headers": {"Content-Type": "text/html"}}


That's all folks